In [1]:
conda install -c bokeh bokeh   #https://docs.bokeh.org/en/2.4.2/docs/reference/bokeh.html

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::jupyterlab==3.0.14=pyhd3eb1b0_1
  - defaults/osx-64::bokeh==2.3.2=py38hecd8cb5_0
  - defaults/noarch::jupyterlab_server==2.4.0=pyhd3eb1b0_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/noarch::nbclassic==0.2.6=pyhd3eb1b0_0
  - defaults/noarch::sphinx==4.0.1=pyhd3eb1b0_0
  - defaults/osx-64::_ipyw_jlab_nb_ext_conf==0.1.0=py38_0
  - defaults/osx-64::anaconda-client==1.7.2=py38_0
  - defaults/noarch::pyls-black==0.4.6=hd3eb1b0_0
  - defaults/noarch::numpydoc==1.1.0=pyhd3eb1b0_1
  - defaults/osx-64::anyio==2.2.0=py38hecd8cb5_1
  - defaults/osx-64::anaconda==2021.05=py38_0
  - defaults/noarch::dask==2021.4.0=pyhd3eb1b0_0
  - defaults/noarch::black==19.10b0=py_0
  - defaults/noarch::anaconda-project==0.9.1=pyhd3eb1b0_1
  - defaults/osx-64::anaconda-navigator==2.3.2=py38hecd8cb5_0
  - defaults/osx-6

In [4]:
conda install -c conda-forge pyproj

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::jupyterlab==3.0.14=pyhd3eb1b0_1
  - defaults/osx-64::bokeh==2.3.2=py38hecd8cb5_0
  - defaults/noarch::jupyterlab_server==2.4.0=pyhd3eb1b0_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/noarch::nbclassic==0.2.6=pyhd3eb1b0_0
  - defaults/noarch::sphinx==4.0.1=pyhd3eb1b0_0
  - defaults/osx-64::_ipyw_jlab_nb_ext_conf==0.1.0=py38_0
  - defaults/osx-64::anaconda-client==1.7.2=py38_0
  - defaults/noarch::pyls-black==0.4.6=hd3eb1b0_0
  - defaults/noarch::numpydoc==1.1.0=pyhd3eb1b0_1
  - defaults/osx-64::anyio==2.2.0=py38hecd8cb5_1
  - defaults/osx-64::anaconda==2021.05=py38_0
  - defaults/noarch::dask==2021.4.0=pyhd3eb1b0_0
  - defaults/noarch::black==19.10b0=py_0
  - defaults/noarch::anaconda-project==0.9.1=pyhd3eb1b0_1
  - defaults/osx-64::anaconda-navigator==2.3.2=py38hecd8cb5_0
  - defaults/osx-6

In [13]:
import numpy as np
import pandas as pd
# Allow Bokeh to display inside Jupyter Notebook
import bokeh.io
# A Bokeh colour pallette for the pie chart
from bokeh.palettes import Category20c
# Various Bokeh plotting and transform functionality
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.transform import cumsum
from math import pi
from bokeh.tile_providers import get_provider, Vendors
# Project from latitude/longitude (EPSG:4326) into Web Mercator (EPSG:385)
from pyproj import Proj

In [6]:
bokeh.io.output_notebook(hide_banner=True)

In [7]:
input_data_housing_df = pd.read_csv("FILE.csv")  #ENTER THE DATASET NEEDED

FileNotFoundError: [Errno 2] No such file or directory: 'california_housing.csv'

In [ ]:
# Scatter plot

p = figure(
    title="Median Income vs Average House Value",
    x_axis_label="Median Income",
    y_axis_label="Average House Value"
)

p.scatter(
    # the x variable
    input_data_housing_df["MedInc"],
    # the y variable
    input_data_housing_df["AverageHouseValue"]
)
# show the visualisation
show(p)

In [ ]:
# Histogram example
# create the figure
p = figure(
    toolbar_location=None,
    title="Distribution of House Age"
)
    
# create the histogram data
hist, edges = np.histogram(
    input_data_housing_df["HouseAge"], 
    density=True, 
    bins=20
)

# display the histogram data
p.quad(
    top=hist, 
    bottom=0, 
    left=edges[:-1], 
    right=edges[1:],
    fill_color="DarkSeaGreen", 
    line_color="white"
)

# show the visualisation
show(p)

In [ ]:
# Map
projection = Proj("EPSG:3857")
mercator = projection(
    input_data_housing_df["Longitude"],
    input_data_housing_df["Latitude"]
)
source = ColumnDataSource(
    data = dict(
        x = mercator[0],
        y = mercator[1]
    )
)
south_east = projection(
    -114.131211, 
    32.534156   
)
north_west = projection(
    -124.409591,
    42.009518
)
p = figure(
    x_range=(south_east[0],north_west[0]),
    y_range=(south_east[1],north_west[1]),
    x_axis_type="mercator", 
    y_axis_type="mercator"
)
p.add_tile("OSM")
p.circle(
    x="x", 
    y="y", 
    size=2, 
    fill_color="blue", 
    fill_alpha=0.5, 
    source=source
)
show(p)

In [ ]:
# Map with hover tool-tips
projection = Proj("EPSG:3857")
mercator = projection(
    input_data_housing_df["Longitude"],
    input_data_housing_df["Latitude"]
)
source = ColumnDataSource(
    data = dict(
        x = mercator[0],
        y = mercator[1],
        aveocc = input_data_housing_df["AveOccup"],
        medinc = input_data_housing_df["MedInc"]
    )
)
south_east = projection(
    -114.131211, 
    32.534156   
)
north_west = projection(
    -124.409591,
    42.009518
)
tooltips = [
    ("Average Occupancy","@aveocc"),
    ("Median Income","@medinc")
]
p = figure(
    x_range=(south_east[0],north_west[0]),
    y_range=(south_east[1],north_west[1]),
    x_axis_type="mercator", 
    y_axis_type="mercator",
    tooltips = tooltips
)
p.add_tile("OSM")
p.circle(
    x="x", 
    y="y", 
    size=2, 
    fill_color="blue", 
    fill_alpha=0.5, 
    source=source
)
show(p)

In [ ]:
# Column chart
input_data_housing_df = pd.read_csv("FILE.csv")    #ENTER THE DATASET NEEDED
counts = input_data_housing_df["gear"].value_counts()
gears = [str(x) for x in counts.index.tolist()]
p = figure(
    x_range=gears,
    height=400, 
    title="Counts of Number of Gears"
)
colours=["DarkSlateGrey"] * len(gears)
colours[0] = "FireBrick"
p.vbar(
    x=gears,
    top=counts, 
    width=0.7,
    color=colours
)
show(p)